## 为什么我们需要程序栈？
- demo

```
// function_example.c
#include <stdio.h>
int static add(int a, int b)
{
    return a+b;
}


int main()
{
    int x = 5;
    int y = 10;
    int u = add(x, y);
}
```

- 编译
```
gcc -g -c function_example.c
objdump -d -M intel -S function_example.o
```

- 汇编代码
```
int static add(int a, int b)
{
   0:   55                      push   rbp
   1:   48 89 e5                mov    rbp,rsp
   4:   89 7d fc                mov    DWORD PTR [rbp-0x4],edi
   7:   89 75 f8                mov    DWORD PTR [rbp-0x8],esi
    return a+b;
   a:   8b 55 fc                mov    edx,DWORD PTR [rbp-0x4]
   d:   8b 45 f8                mov    eax,DWORD PTR [rbp-0x8]
  10:   01 d0                   add    eax,edx
}
  12:   5d                      pop    rbp
  13:   c3                      ret    
0000000000000014 <main>:
int main()
{
  14:   55                      push   rbp
  15:   48 89 e5                mov    rbp,rsp
  18:   48 83 ec 10             sub    rsp,0x10
    int x = 5;
  1c:   c7 45 fc 05 00 00 00    mov    DWORD PTR [rbp-0x4],0x5
    int y = 10;
  23:   c7 45 f8 0a 00 00 00    mov    DWORD PTR [rbp-0x8],0xa
    int u = add(x, y);
  2a:   8b 55 f8                mov    edx,DWORD PTR [rbp-0x8]
  2d:   8b 45 fc                mov    eax,DWORD PTR [rbp-0x4]
  30:   89 d6                   mov    esi,edx
  32:   89 c7                   mov    edi,eax
  34:   e8 c7 ff ff ff          call   0 <add>
  39:   89 45 f4                mov    DWORD PTR [rbp-0xc],eax
  3c:   b8 00 00 00 00          mov    eax,0x0
}
  41:   c9                      leave  
  42:   c3                      ret    
```

- call 指令
  - 跳转后的程序地址

- push、mov 以及 pop、ret
  - 对应压栈（Push）和出栈（Pop）操作

- 在内存中开辟一段空间，用栈（后进先出，LIFO）的数据结构来单独记录跳转回来的地址

![image](lesson07_01.png)

- 真实情况是，压栈的不只有函数调用完成后的返回地址。比如需要传递一些参数时，寄存器不够用的时候，也会被压入栈中。
  - 整个函数 A 所占用的所有内存空间，就是函数 A 的栈帧（Stack Frame）
  - 实际的程序栈布局，顶和底与我们的乒乓球桶相比是倒过来的。
  - 底在最上面，顶在最下面，这样的布局是因为栈底的内存地址是在一开始就固定的。
  - 而一层层压栈之后，栈顶的内存地址是在逐渐变小而不是变大。

![image](lesson07_02.png)

- 详细说明
  - main 函数调用 add 函数时，add 函数入口在 0～1 行，add 函数结束之后在 12～13 行。
  - 在调用第 34 行的 call 指令时，会把当前的 PC 寄存器里的下一条指令的地址压栈，保留函数调用结束后要执行的指令地址。
  - 而 add 函数的第 0 行，push rbp 这个指令，就是在进行压栈。
  - rbp 又叫栈帧指针（Frame Pointer），是一个存放了当前栈帧位置的寄存器。
  - push rbp 就把之前调用函数，也就是 main 函数的栈帧的栈底地址，压到栈顶。
  - 接着，第 1 行的一条命令 mov rbp, rsp 里，则是把 rsp 这个栈指针（Stack Pointer）的值复制到 rbp 里，而 rsp 始终会指向栈顶。
  - 这个命令意味着，rbp 这个栈帧指针指向的地址，变成当前最新的栈顶，也就是 add 函数的栈帧的栈底地址了。
  - 而在函数 add 执行完成之后，又会分别调用第 12 行的 pop rbp 来将当前的栈顶出栈，这部分操作维护好了我们整个栈帧。
  - 然后，可以调用第 13 行的 ret 指令，这时候同时要把 call 调用的时候压入的 PC 寄存器里的下一条指令出栈，更新到 PC 寄存器中，将程序的控制权返回到出栈后的栈顶。

---

## 如何利用函数内联进行性能优化？
- 把一个实际调用的函数产生的指令，直接插入到的位置，来替换对应的函数调用指令。
- 函数内联（Inline）
- 还可以在定义函数的地方，加上 inline 的关键字，来提示编译器对函数进行内联。
  - 内联带来的优化是，CPU 需要执行的指令数变少了，根据地址跳转的过程不需要了，压栈和出栈的过程也不用了。
  - 内联意味着，我们把可以复用的程序指令在调用它的地方完全展开了。如果一个函数在很多地方都被调用了，那么就会展开很多次，整个程序占用的空间就会变大了。
- 没有调用其他函数，只会被调用的函数，我们一般称之为叶子函数（或叶子过程）

![image](lesson07_03.png)

- demo

```
#include <stdio.h>
#include <time.h>
#include <stdlib.h>

int static add(int a, int b)
{
    return a+b;
}

int main()
{
    srand(time(NULL));
    int x = rand() % 5
    int y = rand() % 10;
    int u = add(x, y)
    printf("u = %d\n", u)
}
```

- 编译（加上对应的一个让编译器自动优化的参数 -O）

```
gcc -g -c -O function_example_inline.c
objdump -d -M intel -S function_example_inline.o
```

- 结果

```
    return a+b;
  4c:   01 de                   add    esi,ebx

```

---

## 总结延伸
- 《深入理解计算机系统（第三版）》的 3.7 小节《过程》
- 通过搜索引擎搞清楚 function_example.c 每一行汇编代码的含义，能够帮你进一步深入了解程序栈、栈帧、寄存器以及 Intel CPU 的指令集。

---